In [121]:
from numpy import zeros
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import widgetbox, column
from bokeh.models import Select, CustomJS, ColumnDataSource, Circle, Line, HoverTool, Slider
from openpyxl import load_workbook

In [122]:
output_notebook()

Loading BokehJS ...

In [123]:
# read data from xslx
wb = load_workbook('indicator hiv estimated prevalence% 15-49.xlsx')
sheets = wb.get_sheet_names()
ws = wb.get_sheet_by_name(sheets[0])

In [124]:
# prepare the data set
years = range(1979, 2012)
total_data = {'x': years, 'y': zeros(33)}
for i in range(2,277):
    hiv_data = []
    for cell in ws[str(i)]:
        hiv_data.append(cell.value)
    country = hiv_data[0]
    hiv_data = hiv_data[1:len(hiv_data)]
    total_data[country] = hiv_data
source = ColumnDataSource(data=total_data)

# initial country list
countries = []
for cell in ws['A']:
    countries.append(cell.value)
countries = countries[1:len(countries)]

In [125]:
# add hover tool
hover = HoverTool(
    tooltips=[('Year', '@x'), ('Percent', '@y{%0.2f}%')],
    formatters={'x': 'datetime','y': 'printf'},
    mode='vline'
)

In [126]:
plot = figure(title = "Adults with HIV (%, age 15-49)",
              plot_width = 800,
              plot_height = 400,
              x_axis_label = 'Year',
              y_axis_label = 'Estimated HIV Prevalence %',
              y_range = [0, 28],
              tools = [hover])

plot.circle('x', 'y', source = source)
plot.line('x', 'y', source = source)

GlyphRenderer(id='1bf3a244-73a9-4f64-92aa-0cc50b3aaf13', ...)

In [127]:
# Interaction using javascript callback
callback = CustomJS(args=dict(source=source), code="""
    var country = cb_obj.value;
    var data = source.data;
    x = data['x'];
    y = data['y'];
    for (i = 0; i < x.length; i++) {
        y[i] = data[country][i]
    }
    source.change.emit();
""")

select = Select(title="Country:", value="Abkhazia", options=countries)
select.js_on_change('value', callback)

In [128]:
layout = column(widgetbox(select), plot)
show(layout)